In [1]:
#basic package
import tqdm
import os
import numpy as np
import glob
import pandas as pd
import sys
import time
import datetime as dt
import itertools
import re
import math
from collections import Counter
import operator
from dtaidistance import dtw
import random
from sklearn.preprocessing import normalize
import pickle #to save/load list of selected hens

#plot
import matplotlib.pyplot as plt
import seaborn as sns

#network
import networkx as nx

In [2]:
PACKAGE_PARENT = '../'
SCRIPT_DIR = os.path.dirname(os.path.realpath(os.path.join(os.getcwd(), os.path.expanduser('__file__'))))
sys.path.append(os.path.normpath(os.path.join(SCRIPT_DIR, PACKAGE_PARENT)))
from UTILS import chi2_distance, correct_key
import config_origins as config

C:\Users\camil\anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


2


# Define parameters

In [3]:
print('change the configuration file if not done yet!')
path_extracted_data = config.path_extracted_data
path_initial_data = config.path_initial_data
id_run = config.id_run
#id_run = 'chapter0_final_'
#path_dataoutput = r'G:\VPHI\Welfare\2- Research Projects\OFHE2.OriginsE2\DataOutput'
#path_extracted_data = os.path.join(path_dataoutput,'TrackingSystem') 
#path_extracted_data = os.path.join(path_extracted_data, id_run)
dico_night_hour = config.dico_night_hour
dico_matching = config.dico_matching
li_binmn = config.li_binmn
penalty = config.penalty
dico_window = config.dico_window
path_extracted_data_SNA = config.path_extracted_data_SNA
#create a director if not existing
if not os.path.exists(path_extracted_data_SNA):
    os.makedirs(path_extracted_data_SNA)
print(id_run)

change the configuration file if not done yet!
ALLDATA_


C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Download variables

In [4]:
#daily variables (one row per (henID, date))
#daily_ALL_Variable_Tranformed
df_daily = pd.read_csv(os.path.join(path_extracted_data, id_run+'_daily_ALL_variables.csv'), sep=';',
                     parse_dates=['level'], dayfirst=True) 
#dico of hen as keys and pen as values
dico_h_p = dict(zip(df_daily['HenID'].tolist(), df_daily['PenID'].tolist()))
print(df_daily.shape)
df_daily.head(3)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(42750, 158)


,HenID,level,duration_1_Zone,duration_2_Zone,duration_3_Zone,duration_4_Zone,duration_5_Zone,verification_daily_total_duration,verification_daily_total_nbr_hour,dur_values,...,nbr_transition_at_h21,nbr_transition_at_h22,nbr_transition_at_h23,nbr_transition_next1hafterlightoff,nbr_h_per_day,correct_amount_of_hour,DOA,WOA,DIB,WIB
0,hen_130,2020-10-05,NaN,0.0,0.0,0.0,32400.0,32400.0,9.0,"[0.0, 0.0, 0.0, 0.0, 32400.0]",...,0.0,0.0,0.0,0.0,9,True,124,18,6,1
1,hen_130,2020-10-06,NaN,0.0,780.0,325.0,31295.0,32400.0,9.0,"[0.0, 0.0, 780.0, 325.0, 31295.0]",...,0.0,0.0,0.0,0.0,9,True,125,18,7,1
2,hen_130,2020-10-07,NaN,206.0,249.0,130.0,31815.0,32400.0,9.0,"[0.0, 206.0, 249.0, 130.0, 31815.0]",...,0.0,0.0,0.0,0.0,9,True,126,18,8,2


# Distance across pairs of MLPS from same day with %time in a same zone- for daily SNA

In [5]:
#download the MLPs vectors during day only! as we compare on same day, we can restrict to the exact daily calendar!
dico_pen_level_h = pickle.load(open(os.path.join(path_extracted_data, 
                                                 id_run+'dico_pen_level_h_allzoneidseclevel_DAILYLEVEL.pkl'), 'rb'))

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
#dico of hen as keys and pen as values
dico_h_p = dict(zip(df_daily['HenID'].tolist(), df_daily['PenID'].tolist()))
#df_ = df_daily.groupby('PenID')['HenID'].agg(lambda x: list(set(x))).reset_index()
#dico_penid_lihen = dict(zip(df_['PenID'].tolist(), df_['HenID'].tolist()))
#dico_cla_cla = {'EPI':'other', 'MEXP':'MEXP', 'LEXP':'LEXP', 'NewAfterEpi':'other', 'LEXPLOST':'other', 'MEXPLOST':'other'}
#df_daily['CLASS'] = df_daily['CLASS'].map(lambda x: dico_cla_cla[x])
#dico_h_class = dict(zip(df_daily['HenID'].tolist(), df_daily['CLASS'].tolist()))

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
#max size
(160*159/2)*285

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


3625200.0

In [ ]:
#start recording the time it last
START_TIME = time.perf_counter()
li_df = []
li_date = sorted(set(df_daily['level'].tolist()))
for d in tqdm.tqdm(sorted(list(li_date))):
    #list of hens available that day (i.e. functioning tags)
    li_hen_d = df_daily[df_daily['level']==d]['HenID'].unique()
    #we use symmetric measures, so we only do each combination once
    for i,h1 in enumerate(li_hen_d):
        #lets compute for hen h1 its similarity with all other birds
        for h2 in li_hen_d[i+1:]:
            l1 = np.array(dico_pen_level_h[dico_h_p[h1]][d][h1])
            l2 = np.array(dico_pen_level_h[dico_h_p[h2]][d][h2])
            if len(l1)==len(l2):
                li_df.append({'HenID1':h1,'HenID2':h2,'date':d,
                              'total_nbr_sec_involved':len(l1),'nbr_sec_CommunZone':sum(l1==l2),
                              'PenID1':dico_h_p[h1], 'PenID2':dico_h_p[h2]})
            else: 
                print(e)
                print(h1, h2)
                print(d)
END_TIME = time.perf_counter()
print ("Total running time: %.2f mn" %((END_TIME-START_TIME)/60))
df_prox = pd.DataFrame(li_df)
df_prox['perc_time_communZone'] = df_prox.apply(lambda x: x['nbr_sec_CommunZone']/x['total_nbr_sec_involved']*100, axis=1)
df_prox['hen_pair'] = df_prox.apply(lambda x: '-'.join(sorted([x['HenID1'],x['HenID2']])), axis=1)
df_prox['pen_pair'] = df_prox.apply(lambda x: '-'.join(sorted([x['PenID1'],x['PenID2']])), axis=1)
df_prox.to_csv(os.path.join(path_extracted_data, id_run+'df_proximity_perctime.csv'), index=False, sep=';')
print(df_prox.shape)
df_prox.head(3)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
 16%|████████████                                                               | 46/285 [9:57:13<51:43:00, 779.00s/it]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\camil\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-89f7cf3dfaa9>", line 17, in <module>
    'PenID1':dico_h_p[h1], 'PenID2':dico_h_p[h2]})
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\camil\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2045, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\camil\anaconda3\lib\site-packages\IPython\core\ultratb.py", line 1170, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "C:\Users\camil\anaconda3\lib\site-package

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\camil\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-89f7cf3dfaa9>", line 17, in <module>
    'PenID1':dico_h_p[h1], 'PenID2':dico_h_p[h2]})
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\camil\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2045, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\camil\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3338, in run_ast_nodes
    if (await self.run_code(code, result,  async_=asy)):
  File "C:\Users\camil\anaconda3\lib\site-packages\IPython

In [8]:
END_TIME = time.perf_counter()
print ("Total running time: %.2f mn" %((END_TIME-START_TIME)/60))
df_prox = pd.DataFrame(li_df)
df_prox['perc_time_communZone'] = df_prox.apply(lambda x: x['nbr_sec_CommunZone']/x['total_nbr_sec_involved']*100, axis=1)
df_prox['hen_pair'] = df_prox.apply(lambda x: '-'.join(sorted([x['HenID1'],x['HenID2']])), axis=1)
df_prox['pen_pair'] = df_prox.apply(lambda x: '-'.join(sorted([x['PenID1'],x['PenID2']])), axis=1)
df_prox.to_csv(os.path.join(path_extracted_data, id_run+'df_proximity_perctime.csv'), index=False, sep=';')
print(df_prox.shape)
df_prox.head(3)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Total running time: 597.43 mn
ERROR! Session/line number was not unique in database. History logging moved to new session 1943
(481380, 10)


,HenID1,HenID2,date,total_nbr_sec_involved,nbr_sec_CommunZone,PenID1,PenID2,perc_time_communZone,hen_pair,pen_pair
0,hen_131,hen_134,2020-09-30,28800,19,pen10,pen10,0.065972,hen_131-hen_134,pen10-pen10
1,hen_131,hen_138,2020-09-30,28800,20895,pen10,pen10,72.552083,hen_131-hen_138,pen10-pen10
2,hen_131,hen_139,2020-09-30,28800,1050,pen10,pen10,3.645833,hen_131-hen_139,pen10-pen10


# Distance across pairs of MLPS from same day with Chi2&DTW- for daily SNA

#compute the CHI2DISTANCE & DTW for each bird with all other birds
#start recording the time it last
START_TIME = time.perf_counter()
li_df = []
#df.sort_values(['Timestamp'], inplace=True)
#take (subset of) dates and hens that are available (i.e. for which we comptue the variable on)
#df_daily_h = df_daily[(df_daily['DOA']>=150)&(df_daily['DOA']<=380)].copy()
li_date = set(df_daily['level'].tolist())
li_zone = ['1_Zone', '2_Zone', '3_Zone', '4_Zone', '5_Zone']
for d in tqdm.tqdm(sorted(list(li_date))):
    li_hen_d = df_daily[df_daily['level']==d]['HenID'].unique()
    #we use symmetric measures, so we only do each combination once
    for i,h1 in enumerate(li_hen_d):

        #lets compute for hen h1 its difference with all other birds
        for h2 in li_hen_d[i+1:]:
            
            try:
                ############ Overall aviary usage similarity across days
                l1_chi2 = df_daily[(df_daily['HenID']==h1)&(df_daily['level']==d)]['dur_values_normalized'].values[0]
                l2_chi2 = df_daily[(df_daily['HenID']==h2)&(df_daily['level']==d)]['dur_values_normalized'].values[0]
                dico_ = {'date':d, 'henId':h1, 'henID2':h2, 'chi2distance':chi2_distance(l1_chi2,l2_chi2, remove_warning=True)}
                #'dur_values_normalized_henId':l1_chi2,'dur_values_normalized_henID2':l2_chi2,

                ############# Specifics' zones usage similarity across days
                for nbr_binmn in li_binmn:
                    #ALL zone together with chi2distance
                    l1_chi2 = dico_pen_bin_level_h[dico_h_p[h1]][nbr_binmn][d][h1]
                    l2_chi2 = dico_pen_bin_level_h[dico_h_p[h2]][nbr_binmn][d][h2]
                    dico_['nbr_obs_chi2all_'+str(nbr_binmn)] = len(l1_chi2)
                    li_chi2 = [chi2_distance(l1_chi2[i],l2_chi2[i], remove_warning=True) for i in range(0,len(l1_chi2))]
                    dico_['li_chi2_'+str(nbr_binmn)] = li_chi2
                    dico_['chi2distance_ALL_'+str(nbr_binmn)] = np.mean(li_chi2)

                    #per zone with DTW
                    for ZONE in li_zone:
                        dtw_value = np.nan
                        try:
                            l1_dtw = np.array(dico_pen_bin_zone_level_h[dico_h_p[h1]][nbr_binmn][ZONE][d][h1], dtype=np.double)
                            l2_dtw = np.array(dico_pen_bin_zone_level_h[dico_h_p[h2]][nbr_binmn][ZONE][d][h2], dtype=np.double)
                            #compute distance measure
                            #psi: #of start and end points of a sequence can be ignored if this would lead to a lower distance
                            dtw_value, paths = dtw.warping_paths(l1_dtw, l2_dtw, window=dico_window[nbr_binmn], psi=0, penalty=penalty)
                        except Exception as e:
                            print(e)
                            sys.exit()
                            pass #dont do anything
                        dico_['DTW_'+str(nbr_binmn)+'_'+ZONE] = round(dtw_value,1)  
                        dico_['nbr_obs_l1_'+str(nbr_binmn)+'_'+ZONE] = len(l1_dtw)
                        dico_['nbr_obs_l2_'+str(nbr_binmn)+'_'+ZONE] = len(l2_dtw)
                li_df.append(dico_)
            except Exception as e:
                print(e)
                print(h1, h2)
                print(d)
                
END_TIME = time.perf_counter()
print ("Total running time: %.2f mn" %((END_TIME-START_TIME)/60))
#turn inter and intra distances into a dataframe                
df_inter = pd.DataFrame(li_df)
#df_inter = df_inter[~df_inter['chi2distance'].isnull()]
df_inter['hen_pair'] = df_inter.apply(lambda x: '-'.join(sorted([x['henId'],x['henID2']])), axis=1)
df_inter['DTW_30_all'] = df_inter[['DTW_30_1_Zone', 'DTW_30_2_Zone', 'DTW_30_3_Zone','DTW_30_4_Zone','DTW_30_5_Zone']].sum(axis=1)
li_col = list(df_inter.columns)
li_remove = [i for i in li_col if i.startswith('li_chi2_')]
print(li_remove)
li_keep = [i for i in li_col if i not in li_remove]
#df_inter.iloc[0:1000].to_csv(os.path.join(path_extracted_data_visual_adap, id_run+'_df_DistanceBetweenHenSim_4verification.csv'), sep=';', index=False)
#df_inter.filter(li_keep).to_csv(os.path.join(path_extracted_data_visual_adap ,id_run+'_df_DistanceBetweenHenSim.csv'), sep=';', index=False)
df_inter.to_csv(os.path.join(path_extracted_data ,id_run+'_df_DistanceBetweenHenSim_DAILYLEVEL.csv'), sep=';', index=False)
print(df_inter.shape)
display(df_inter.head(3))

In [7]:
print(df_inter.shape)
display(df_inter.head(3))

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



ERROR! Session/line number was not unique in database. History logging moved to new session 1942
Traceback (most recent call last):
  File "C:\Users\camil\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-52bb76da6181>", line 1, in <module>
    print(df_inter.shape)
NameError: name 'df_inter' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\camil\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2045, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\camil\anaconda3\lib\site-packages\IPython\core\ultratb.py", line 1170, in get_records
    return _fixed_getinnerframes(etb, n

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\camil\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-52bb76da6181>", line 1, in <module>
    print(df_inter.shape)
NameError: name 'df_inter' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\camil\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2045, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\camil\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3338, in run_ast_nodes
    if (await self.run_code(code, result,  async_=asy)):
  File "C:\Users\camil\anaconda3\lib\site-packages\IPython\core\inte

TypeError: object of type 'NoneType' has no len()

# Distance across any pairs of MLPS (i.e. any days) - for daily clusterID

#download MLPs vectors
#For now we restrict to 2h-17h in order to always compare same length TS as we will compare across days, the lenght might differ
dico_pen_bin_level_h = pickle.load(open(os.path.join(path_extracted_data_visual_adap, 
                                                     id_run+'dico_pen_bin_level_h_2h-17h59LEVEL.pkl'), 'rb'))

#compute mvt distances between any two MLPs on any day from any pen
dico_hendate_mvt = {}
for p,dico_bin_level_h in dico_pen_bin_level_h.items():
    for level, dico_hen_mvt in dico_bin_level_h[15].items():
        if level!=dt.datetime(2020,9,29):
            for henID,limvt in dico_hen_mvt.items():
                dico_hendate_mvt[henID+'/'+str(level).split(' ')[0]] = limvt
dico_hendate_mvt = OrderedDict(dico_hendate_mvt)
li_hendate = list(dico_hendate_mvt.keys())
li_mvt = list(dico_hendate_mvt.values())
len(dico_hendate_mvt)

li_df = []
for i in tqdm.tqdm(range(0,len(li_mvt))):
    li_mvt1 = li_mvt[i]
    for j in range(i+1,len(li_mvt)):
        dico_ = {}
        li_mvt2 = li_mvt[j]
        li_chi2 = [chi2_distance(li_mvt1[i],li_mvt2[i]) for i in range(0,len(li_mvt1))]
        dico_['chi2distance_ALL_15'] = np.mean(li_chi2)
        dico_['h1'] = li_hendate[i]
        dico_['h2'] = li_hendate[j]
        li_df.append(dico_)

df_anypair = pd.DataFrame(li_df)
display(df_anypair[df_anypair['chi2distance_ALL_15'].isnull()])
df_anypair['henID1'] = df_anypair['h1'].map(lambda x: x.split('/')[0])
df_anypair['henID2'] = df_anypair['h2'].map(lambda x: x.split('/')[0])
df_anypair['level1'] = df_anypair['h1'].map(lambda x: dt.datetime.strptime(x.split('/')[1], '%Y-%m-%d'))
df_anypair['level2'] = df_anypair['h2'].map(lambda x: dt.datetime.strptime(x.split('/')[1], '%Y-%m-%d'))
#df_interintra['weeks_in_laying_barn'] = df_interintra['DOA'].map(lambda x: int((x-119)/7)+1)
#df_interintra['hen_pair'] = df_interintra.apply(lambda x: '-'.join(sorted([x['henID1'],x['henID2']])), axis=1)
df_anypair.to_csv(os.path.join(path_extracted_data_visual_adap ,id_run+'_df_DistanceBetweenAnyDaysHenSim_2h-17h59LEVEL.csv'), sep=';', index=False)
print(df_anypair.shape)
display(df_anypair.head(3))

plt.hist(df_anypair['chi2distance_ALL_15']);

l1 = np.array(['ad','ad','er','ww'])
l2 = np.array(['ad','ed','er','ww'])
sum(l1==l2)